In [1]:
import functools

import numpy as np
import pennylane as qml
from scipy.linalg import expm

from graddft_qnn.dft_qnn import DFTQNN
from graddft_qnn.unitary_rep import O_h

# Defining...

## Unitary reps

In [2]:
theta = 1.0

In [3]:
unitary_reps = O_h._180_deg_rot()
unitary_reps_3_axis = O_h._180_deg_rot_3_axis()
unitary_reps_rot_ref = O_h._180_deg_rot_ref()

## Ansatz

In [4]:
XII = functools.reduce(np.kron, [qml.X.compute_matrix(), np.eye(2), np.eye(2)])
IXI = functools.reduce(np.kron, [np.eye(2), qml.X.compute_matrix(), np.eye(2)])
IIX = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.X.compute_matrix()])

In [5]:
YII = functools.reduce(np.kron, [qml.Y.compute_matrix(), np.eye(2), np.eye(2)])
IYI = functools.reduce(np.kron, [np.eye(2), qml.Y.compute_matrix(), np.eye(2)])
IIY = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Y.compute_matrix()])

In [6]:
ZII = functools.reduce(np.kron, [qml.Z.compute_matrix(), np.eye(2), np.eye(2)])
IZI = functools.reduce(np.kron, [np.eye(2), qml.Z.compute_matrix(), np.eye(2)])
IIZ = functools.reduce(np.kron, [np.eye(2), np.eye(2), qml.Z.compute_matrix()])

$$
ZZ(\phi) = \exp\left(-i \frac{\phi}{2} (Z \otimes Z)\right) =
\begin{bmatrix}
    e^{-i \phi / 2} & 0 & 0 & 0 \\
    0 & e^{i \phi / 2} & 0 & 0 \\
    0 & 0 & e^{i \phi / 2} & 0 \\
    0 & 0 & 0 & e^{-i \phi / 2}
\end{bmatrix}
$$

$ZZ(0) = I$

In [7]:
_ZZZ = functools.reduce(
    np.kron,
    [qml.Z.compute_matrix(), qml.Z.compute_matrix(), qml.Z.compute_matrix()],
)
ZZZ_gen = -1j * theta / 2 * _ZZZ

expm(-1 theta X) exmp(-1 theta Z) -> measurement XZ
any output of the twirling is measurement, and generator

In [8]:
ZZZ = expm(ZZZ_gen)

In [9]:
_XYZ = functools.reduce(
    np.kron,
    [qml.X.compute_matrix(), qml.Y.compute_matrix(), qml.Z.compute_matrix()],
)
XYZ_gen = -1j * theta / 2 * _XYZ

XYZ = expm(XYZ_gen)

In [10]:
def process(gate_matrix, u_reprs: list[np.array], return_gen=False):
    gen = DFTQNN.twirling(gate_matrix, unitary_reps=u_reprs)
    if isinstance(gen, np.ndarray):
        lcu = qml.pauli_decompose(
            gen, check_hermitian=False, hide_identity=False, pauli=True
        )
        return (lcu, gen) if return_gen else lcu
    return None, gen if return_gen else None

# Run the twirling + generator

In [11]:
[
    process(XII, unitary_reps),
    process(IXI, unitary_reps),
    process(IIX, unitary_reps),
    process(YII, unitary_reps),
    process(IYI, unitary_reps),
    process(IIY, unitary_reps),
    process(ZII, unitary_reps),
    process(IZI, unitary_reps),
    process(IIZ, unitary_reps),
]

[(1+0j) * X(0),
 (1+0j) * X(1),
 (1+0j) * X(2),
 (None, None),
 (None, None),
 (1+0j) * Y(2),
 (None, None),
 (None, None),
 (1+0j) * Z(2)]

In [12]:
[
    process(XII, unitary_reps_3_axis),
    process(IXI, unitary_reps_3_axis),
    process(IIX, unitary_reps_3_axis),
    process(YII, unitary_reps_3_axis),
    process(IYI, unitary_reps_3_axis),
    process(IIY, unitary_reps_3_axis),
    process(ZII, unitary_reps_3_axis),
    process(IZI, unitary_reps_3_axis),
    process(IIZ, unitary_reps_3_axis),
]

[(1+0j) * X(0),
 (1+0j) * X(1),
 (1+0j) * X(2),
 (-0.3333333432674408+0j) * Y(0),
 (-0.3333333432674408+0j) * Y(1),
 (-0.3333333432674408+0j) * Y(2),
 (-0.3333333432674408+0j) * Z(0),
 (-0.3333333432674408+0j) * Z(1),
 (-0.3333333432674408+0j) * Z(2)]

In [13]:
[
    process(XII, unitary_reps_rot_ref),
    process(IXI, unitary_reps_rot_ref),
    process(IIX, unitary_reps_rot_ref),
    process(YII, unitary_reps_rot_ref),
    process(IYI, unitary_reps_rot_ref),
    process(IIY, unitary_reps_rot_ref),
    process(ZII, unitary_reps_rot_ref),
    process(IZI, unitary_reps_rot_ref),
    process(IIZ, unitary_reps_rot_ref),
]

[(1+0j) * X(0),
 (1+0j) * X(1),
 (1+0j) * X(2),
 (-0.5+0j) * Y(0),
 (None, None),
 (None, None),
 (-0.5+0j) * Z(0),
 (None, None),
 (None, None)]

In [14]:
zzz_pauli_gen, zzz_gen = process(ZZZ, unitary_reps_3_axis, True)

In [15]:
expm((-0.20000000298023224 + 0j) * qml.Y(0).compute_matrix())

array([[1.02006676+0.j        , 0.        +0.20133601j],
       [0.        -0.20133601j, 1.02006676+0.j        ]])

In [16]:
zzz_pauli_gen

(0.8775825500488281+0j) * I
+ -0.4794255495071411j * Z(0) @ Z(1) @ Z(2)

In [17]:
zzz = expm(zzz_gen)

In [18]:
zzz

array([[2.13393-1.1093895j, 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       ],
       [0.     +0.j       , 2.13393+1.1093895j, 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       ],
       [0.     +0.j       , 0.     +0.j       , 2.13393+1.1093895j,
        0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       ],
       [0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
        2.13393-1.1093895j, 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       ],
       [0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
        0.     +0.j       , 2.13393+1.1093895j, 0.     +0.j       ,
        0.     +0.j       , 0.     +0.j       ],
       [0.     +0.j       , 0.     +0.j       , 0.     +0.j       ,
       

In [19]:
diag = [zzz[i][i] for i in range(len(zzz))]

In [20]:
assert np.allclose(np.diag(diag), zzz)

In [21]:
diag

[(2.13393-1.1093895j),
 (2.13393+1.1093895j),
 (2.13393+1.1093895j),
 (2.13393-1.1093895j),
 (2.13393+1.1093895j),
 (2.13393-1.1093895j),
 (2.13393-1.1093895j),
 (2.13393+1.1093895j)]

In [22]:
(1 + 2j) * (-1j)

(2-1j)

In [23]:
process(XYZ, unitary_reps_rot_ref)

(0.8775825500488281+0j) * I

# Design a circuit

In [24]:
expm(-1j * 2 * qml.X.compute_matrix())

array([[-0.41614684+0.j        ,  0.        -0.90929743j],
       [ 0.        -0.90929743j, -0.41614684+0.j        ]])

In [25]:
qml.RX.compute_matrix(4)

array([[-0.41614684+0.j        ,  0.        -0.90929743j],
       [ 0.        -0.90929743j, -0.41614684+0.j        ]])

In [40]:
def twirling_(ansatz: np.array, unitary_reps: list[np.array]):
    generator = np.zeros_like(ansatz, dtype=np.complex64)
    ansatz = ansatz.astype(np.complex64)
    for unitary_rep in unitary_reps:
        twirled = 0.5 * (ansatz + unitary_rep @ ansatz @ unitary_rep.conjugate())
        if np.allclose(twirled, np.zeros_like(twirled)):
            print("All zero")
        else:
            print(twirled)
            print(qml.pauli_decompose(twirled))
        print()

In [27]:
def commute_(A: np.array, B: np.array):
    return np.allclose(A @ B, B @ A)

In [28]:
for u in unitary_reps_3_axis:
    assert commute_(ZZZ, u)

In [37]:
ZZZ

array([[0.87758256-0.47942554j, 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.87758256+0.47942554j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.87758256+0.47942554j, 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.87758256-0.47942554j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0

In [41]:
all_gates = [XII, IXI, IIX, YII, IYI, IIY, ZII, IZI, IIZ]
for i, gate in enumerate(all_gates):
    print(i)
    twirling_(gate, unitary_reps_3_axis)
    print("====")

0
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
1.0 * (X(0) @ I(1) @ I(2))

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
1.0 * (X(0) @ I(1) @ I(2

In [54]:
a = qml.X(0) @ qml.Y(0)
qml.matrix(a)

array([[0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j]])

array([[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [59]:
qml.MultiRZ.compute_matrix(1, 4).shape

(16, 16)